## <b><font color='darkblue'>Preface</font></b>
([article source](https://machinelearningmastery.com/building-graph-rag-system-step-by-step-approach/?fbclid=IwZXh0bgNhZW0CMTEAAR1la-9zv31c6hYrmyJ59_9ttnj6szllkChoiF7lhF5hD2QvGFiFRAURyxw_aem_aw-qnM-4W1WghDc9rZwcTg)) <b><font size='3ptx'>Graph RAG, Graph RAG, Graph RAG! This term has become the talk of the town, and you might have come across it as well.</font> But what exactly is Graph RAG, and what has made it so popular? In this article, we’ll explore the concept behind Graph RAG, why it’s needed, and, as a bonus, we’ll discuss how to implement it using LlamaIndex. Let’s get started!</b>

<b>First, let’s address the shift from large language models (LLMs) to Retrieval-Augmented Generation (RAG) systems. <font color='darkred'>LLMs rely on static knowledge, which means they only use the data they were trained on. This limitation often makes them prone to hallucinations—generating incorrect or fabricated information</font>.</b>  To handle this, RAG systems were developed. Unlike LLMs, <b>RAG retrieves data in real-time from external knowledge bases, using this fresh context to generate more accurate and relevant responses. These traditional RAG systems work by using text embeddings to retrieve specific information. While powerful, they come with limitations</b>.

If you’ve worked on RAG-related projects, you’ll probably relate to this: <b>the quality of the system’s response heavily depends on the clarity and specificity of the query. <font color='darkred'>But an even bigger challenge emerged — the inability to reason effectively across multiple documents</font></b>.

### <b><font color='darkgreen'>Tradition RAG problem</font></b>
Now, What does that mean? Let’s take an example. Imagine you’re asking the system:
> “Who were the key contributors to the discovery of DNA’s double-helix structure, and what role did Rosalind Franklin play?”

In a traditional RAG setup, the system might retrieve the following pieces of information:
1. **Document 1:** “James Watson and Francis Crick proposed the double-helix structure in 1953.”
2. **Document 2:** “Rosalind Franklin’s X-ray diffraction images were critical in identifying DNA’s helical structure.”
3. **Document 3:** “Maurice Wilkins shared Franklin’s images with Watson and Crick, which contributed to their discovery.”

The problem? **Traditional RAG systems treat these documents as <font color='darkblue'>independent units</font>.** They don’t connect the dots effectively, leading to fragmented responses like:
> “Watson and Crick proposed the structure, and Franklin’s work was important.”

This response lacks depth and misses key relationships between contributors. <b>Enter Graph RAG! By organizing the retrieved data as a graph, Graph RAG represents each document or fact as a node, and the relationships between them as edges.</b>

### <b><font color='darkgreen'>Basic of Graph RAG</font></b>
Here’s how Graph RAG would handle the same query:
* **Nodes:** Represent facts (e.g., “Watson and Crick proposed the structure,” “Franklin contributed critical X-ray images”).
* **Edges:** Represent relationships (e.g., “Franklin’s images → shared by Wilkins → influenced Watson and Crick”).


By reasoning across these interconnected nodes, Graph RAG can produce a complete and insightful response like:
> “The discovery of DNA’s double-helix structure in 1953 was primarily led by James Watson and Francis Crick. However, this breakthrough heavily relied on Rosalind Franklin’s X-ray diffraction images, which were shared with them by Maurice Wilkins.”

<b>This ability to combine information from multiple sources and answer broader, more complex questions is what makes Graph RAG so popular.</b>

## <b><font color='darkblue'>The Graph RAG</font></b>
<b><font size='3ptx'>We’ll now explore the Graph RAG pipeline, as presented in the paper “[From Local to Global: A Graph RAG Approach to Query-Focused Summarization](https://arxiv.org/pdf/2404.16130)” by Microsoft Research.</font></b>

![graph rag flow](images/1.jpg)

### <b><font color='darkgreen'>Pipeline</font></b>

#### <b><font size='3ptx'>Step 1: Source Documents → Text Chunks</font></b>
LLMs can handle only a limited amount of text at a time. To maintain accuracy and ensure that nothing important is missed, we will first <b>break down large documents into smaller, manageable “chunks” of text for processing</b>.

#### <b><font size='3ptx'>Step 2: Text Chunks → Element Instances</font></b>
From each chunk of source text, we will <b>prompt the LLMs to identify graph nodes and edges</b>. For example, from a news article, the LLMs might detect that “NASA launched a spacecraft” and link “NASA” (`entity: node`) to “spacecraft” (`entity: node`) through “launched” (`relationship: edge`).

#### <b><font size='3ptx'>Step 3: Element Instances → Element Summaries</font></b>
<b>After identifying the elements, the next step is to summarize them into concise, meaningful descriptions using LLMs</b>. This process makes the data easier to understand. For example, for the node “NASA,” the summary could be: “NASA is a space agency responsible for space exploration missions.” For the edge connecting “NASA” and “spacecraft,” the summary might be: “NASA launched the spacecraft in 2023.” These summaries ensure the graph is both rich in detail and easy to interpret.

#### <b><font size='3ptx'>Step 4: Element Summaries → Graph Communities</font></b>
<b>The graph created in the previous steps is often too large to analyze directly. To simplify it, the graph is divided into communities using specialized algorithms like [Leiden](https://en.wikipedia.org/wiki/Leiden_algorithm)</b>. These communities help identify clusters of closely related information. For example, one community might focus on “Space Exploration,” grouping nodes such as “NASA,” “Spacecraft,” and “Mars Rover.” Another might focus on “Environmental Science,” grouping nodes like “Climate Change,” “Carbon Emissions,” and “Sea Levels.” This step makes it easier to identify themes and connections within the dataset.

#### <b><font size='3ptx'>Step 5: Graph Communities → Community Summaries</font></b>
<b>LLMs prioritize important details and fit them into a manageable size. Therefore, each community is summarized to give an overview of the information it contains</b>. For example: A community about “space exploration” might summarize key missions, discoveries, and organizations like NASA or SpaceX. These summaries are useful for answering general questions or exploring broad topics within the dataset.

#### <b><font size='3ptx'>Step 6: Community Summaries → Community Answers → Global Answer</font></b>
Finally, the community summaries are used to answer user queries. Here’s how:
1. **Query the Data**: A user asks, “`What are the main impacts of climate change?`”
2. **Community Analysis**: The AI reviews summaries from relevant communities.
3. **Generate Partial Answers**: Each community provides partial answers, such as:
   - “Rising sea levels threaten coastal cities.”
   - “Disrupted agriculture due to unpredictable weather.”
4. **Combine into a Global Answer**: These partial answers are combined into one comprehensive response
   > “Climate change impacts include rising sea levels, disrupted agriculture, and an increased frequency of natural disasters.”

This process ensures the final answer is detailed, accurate, and easy to understand.

## <b><font color='darkblue'>Step-by-Step Implementation of GraphRAG with LlamaIndex</font></b>
You can build your custom Python implementation or use frameworks like [**LangChain**](https://python.langchain.com/docs/introduction/) or [**LlamaIndex**](https://docs.llamaindex.ai/en/stable/). **For this article, we will use the LlamaIndex baseline code provided on their [**website**](https://docs.llamaindex.ai/en/stable/examples/cookbooks/GraphRAG_v1/); however, I will explain it in a beginner-friendly manner. Additionally, I encountered a parsing problem with the original code, which I will explain later along with how I solved it**.

### <b><font color='darkgreen'>Step 1: Install Dependencies</font></b>
Install the required libraries for the pipeline:
```shell
$ pip install llama-index graspologic numpy==1.24.4 scipy==1.12.0
```

**<font color='orange'>Note</font>**: graspologic: Used for graph algorithms like Hierarchical Leiden for community detection.

In [2]:
!pip freeze | grep -P '(llama-index|graspologic|numpy|scipy)'

graspologic==3.3.0
graspologic-native==1.2.1
llama-index==0.12.3
llama-index-agent-openai==0.4.0
llama-index-cli==0.4.0
llama-index-core==0.12.3
llama-index-embeddings-openai==0.3.1
llama-index-indices-managed-llama-cloud==0.6.3
llama-index-legacy==0.9.48.post4
llama-index-llms-openai==0.3.2
llama-index-multi-modal-llms-openai==0.3.0
llama-index-program-openai==0.3.1
llama-index-question-gen-openai==0.3.0
llama-index-readers-file==0.4.1
llama-index-readers-llama-parse==0.4.0
numpy==1.24.4
scipy==1.12.0


### <b><font color='darkgreen'>Step 2: Load and Preprocess Data</font></b>
<b>Load sample news data, which will be chunked into smaller parts for easier processing</b>. For demonstration, we limit it to 50 samples. Each row (title and text) is converted into a Document object.

In [6]:
import os
import pandas as pd
from llama_index.core import Document

SOURCE_DOCS_PATH = os.path.expanduser('~/Github/bt_test_common/docs/')

doc_files = [os.path.join(SOURCE_DOCS_PATH, fn) for fn in os.listdir(os.path.expanduser(SOURCE_DOCS_PATH)) if fn.endswith('.md')]

In [7]:
doc_files

['/usr/local/google/home/johnkclee/Github/bt_test_common/docs/contributing.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/profiles_hfp_facade.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/background_knowldge_bluetooth.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/utils_bds_broker.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/ReleaseSteps.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/wifi_utils_usages.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/bt_utils_usages.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/general_utils_usages.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/background_knowledge_mobly.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/utils_log_parser.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_common/docs/code-of-conduct.md',
 '/usr/local/google/home/johnkclee/Github/bt_test_commo

In [9]:
def read_doc(file_path):
    file_content = open(file_path, 'r').read()
    return f'Source: {file_path}\n{file_content}'
    
# Convert data into LlamaIndex Document objects
documents = [
    Document(text=read_doc(fn))
    for fn in doc_files
]

In [10]:
documents[0]

Document(id_='168ca918-a8a6-49aa-bf86-59c6bad5eb19', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="Source: /usr/local/google/home/johnkclee/Github/bt_test_common/docs/contributing.md\n# How to Contribute\n\nWe would love to accept your patches and contributions to this project.\n\n## Before you begin\n\n### Sign our Contributor License Agreement\n\nContributions to this project must be accompanied by a\n[Contributor License Agreement](https://cla.developers.google.com/about) (CLA).\nYou (or your employer) retain the copyright to your contribution; this simply\ngives us permission to use and redistribute your contributions as part of the\nproject.\n\nIf you or your current employer have already signed the Google CLA (even if it\nwas for a different project), you probably don't need to do it again.\n\nVisit <https://cla.developers.google.com/> to see your cu

### <b><font color='darkgreen'>Step 3: Split Text into Nodes</font></b>
Use [**SentenceSplitter**](https://docs.llamaindex.ai/en/v0.10.19/api/llama_index.core.node_parser.SentenceSplitter.html) to break down documents into manageable chunks.

In [11]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

<b><font color='orange'>Notes</font></b>: `chunk_overlap=20`: Ensures chunks overlap slightly to avoid missing information at the boundaries

### <b><font color='darkgreen'>Step 4: Configure the LLM, Prompt, and GraphRAG Extractor</font></b>
Set up the LLM (<font color='brown'>e.g., GPT-4</font>). This LLM will later analyze the chunks to extract entities and relationships.

In [16]:
from typing import Any
from dotenv import load_dotenv, find_dotenv
from llama_index.llms.openai import OpenAI

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
llm = OpenAI(model="gpt-4")

<b>The <font color='blue'>GraphRAGExtractor</font> uses the above LLM, a prompt template to guide the extraction process, and a parsing function to process the LLM’s output into structured data.</b> Text chunks (<font color='brown'>called `nodes`</font>) are fed into the extractor. <b>For each chunk, the extractor sends the text to the LLM along with the prompt, which instructs the LLM to identify entities, their types, and their relationships</b>. The response is parsed by a function (**`parse_fn`**), which extracts the entities and relationships. 

These are then converted into <b><font color='blue'>EntityNode</font></b> objects (for entities) and <b><font color='blue'>Relation</font></b> objects (for relationships), with descriptions stored as metadata. <b>The extracted entities and relationships are saved into the text chunk’s metadata, ready for use in building knowledge graphs or performing queries</b>.

<b><font color='orange'>Notes:</font></b>
> The issue in the original implementation was that the **`parse_fn`** failed to extract entities and relationships from the LLM-generated response, resulting in empty outputs for parsed entities and relationships. This occurred due to overly complex and rigid regular expressions that did not align well with the LLM response’s actual structure, particularly regarding inconsistent formatting and line breaks in the output. To address this, I have simplified the parse_fn by replacing the original regex patterns with straightforward patterns designed to match the key-value structure of the LLM response more reliably. The updated part looks like this:

In [17]:
entity_pattern = r'entity_name:\s*(.+?)\s*entity_type:\s*(.+?)\s*entity_description:\s*(.+?)\s*'
relationship_pattern = r'source_entity:\s*(.+?)\s*target_entity:\s*(.+?)\s*relation:\s*(.+?)\s*relationship_description:\s*(.+?)\s*'

def parse_fn(response_str: str) -> Any:
    entities = re.findall(entity_pattern, response_str)
    relationships = re.findall(relationship_pattern, response_str)
    return entities, relationships

The prompt template and <b><font color='blue'>GraphRAGExtractor</font></b> class are kept as is, as follows:

In [18]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

from typing import Any, List, Callable, Optional, Union, Dict
from IPython.display import Markdown, display

from llama_index.core.async_utils import run_jobs
from llama_index.core.indices.property_graph.utils import (
    default_parse_triplets_fn,
)
from llama_index.core.graph_stores.types import (
    EntityNode,
    KG_NODES_KEY,
    KG_RELATIONS_KEY,
    Relation,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.default_prompts import (
    DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
)
from llama_index.core.schema import TransformComponent, BaseNode
from llama_index.core.bridge.pydantic import BaseModel, Field


class GraphRAGExtractor(TransformComponent):
    """Extract triples from a graph.

    Uses an LLM and a simple prompt + output parsing to extract paths (i.e. triples) and entity, relation descriptions from text.

    Args:
        llm (LLM):
            The language model to use.
        extract_prompt (Union[str, PromptTemplate]):
            The prompt to use for extracting triples.
        parse_fn (callable):
            A function to parse the output of the language model.
        num_workers (int):
            The number of workers to use for parallel processing.
        max_paths_per_chunk (int):
            The maximum number of paths to extract per chunk.
    """

    llm: LLM
    extract_prompt: PromptTemplate
    parse_fn: Callable
    num_workers: int
    max_paths_per_chunk: int

    def __init__(
        self,
        llm: Optional[LLM] = None,
        extract_prompt: Optional[Union[str, PromptTemplate]] = None,
        parse_fn: Callable = default_parse_triplets_fn,
        max_paths_per_chunk: int = 10,
        num_workers: int = 4,
    ) -> None:
        """Init params."""
        from llama_index.core import Settings

        if isinstance(extract_prompt, str):
            extract_prompt = PromptTemplate(extract_prompt)

        super().__init__(
            llm=llm or Settings.llm,
            extract_prompt=extract_prompt or DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
            parse_fn=parse_fn,
            num_workers=num_workers,
            max_paths_per_chunk=max_paths_per_chunk,
        )

    @classmethod
    def class_name(cls) -> str:
        return "GraphExtractor"

    def __call__(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes."""
        return asyncio.run(
            self.acall(nodes, show_progress=show_progress, **kwargs)
        )

    async def _aextract(self, node: BaseNode) -> BaseNode:
        """Extract triples from a node."""
        assert hasattr(node, "text")

        text = node.get_content(metadata_mode="llm")
        try:
            llm_response = await self.llm.apredict(
                self.extract_prompt,
                text=text,
                max_knowledge_triplets=self.max_paths_per_chunk,
            )
            entities, entities_relationship = self.parse_fn(llm_response)
        except ValueError:
            entities = []
            entities_relationship = []

        existing_nodes = node.metadata.pop(KG_NODES_KEY, [])
        existing_relations = node.metadata.pop(KG_RELATIONS_KEY, [])
        metadata = node.metadata.copy()
        for entity, entity_type, description in entities:
            metadata[
                "entity_description"
            ] = description  # Not used in the current implementation. But will be useful in future work.
            entity_node = EntityNode(
                name=entity, label=entity_type, properties=metadata
            )
            existing_nodes.append(entity_node)

        metadata = node.metadata.copy()
        for triple in entities_relationship:
            subj, rel, obj, description = triple
            subj_node = EntityNode(name=subj, properties=metadata)
            obj_node = EntityNode(name=obj, properties=metadata)
            metadata["relationship_description"] = description
            rel_node = Relation(
                label=rel,
                source_id=subj_node.id,
                target_id=obj_node.id,
                properties=metadata,
            )

            existing_nodes.extend([subj_node, obj_node])
            existing_relations.append(rel_node)

        node.metadata[KG_NODES_KEY] = existing_nodes
        node.metadata[KG_RELATIONS_KEY] = existing_relations
        return node

    async def acall(
        self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any
    ) -> List[BaseNode]:
        """Extract triples from nodes async."""
        jobs = []
        for node in nodes:
            jobs.append(self._aextract(node))

        return await run_jobs(
            jobs,
            workers=self.num_workers,
            show_progress=show_progress,
            desc="Extracting paths from text",
        )

In [20]:
KG_TRIPLET_EXTRACT_TMPL = """
-Goal-
Given a text document, identify all entities and their entity types from the text and all relationships among the identified entities.
Given the text, extract up to {max_knowledge_triplets} entity-relation triplets.

-Steps-
1. Identify all entities. For each identified entity, extract the following information:
- entity_name: Name of the entity, capitalized
- entity_type: Type of the entity
- entity_description: Comprehensive description of the entity's attributes and activities
Format each entity as ("entity")

2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relation: relationship between source_entity and target_entity
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other

Format each relationship as ("relationship")

3. When finished, output.

-Real Data-
######################
text: {text}
######################
output:"""

In [21]:
kg_extractor = GraphRAGExtractor(
    llm=llm,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL,
    max_paths_per_chunk=2,
    parse_fn=parse_fn,
)

### <b><font color='darkgreen'>Step 5: Build the Graph Index</font></b>
The <b><font color='blue'>PropertyGraphIndex</font></b> extracts entities and relationships from text using **kg_extractor** and stores them as nodes and edges in the <b><font color='blue'>GraphRAGStore</font></b>.

In [23]:
import re
from llama_index.core.graph_stores import SimplePropertyGraphStore
import networkx as nx
from graspologic.partition import hierarchical_leiden
from llama_index.core.llms import ChatMessage


class GraphRAGStore(SimplePropertyGraphStore):
    community_summary = {}
    max_cluster_size = 5

    def generate_community_summary(self, text):
        """Generate summary for a given text using an LLM."""
        messages = [
            ChatMessage(
                role="system",
                content=(
                    "You are provided with a set of relationships from a knowledge graph, each represented as "
                    "entity1->entity2->relation->relationship_description. Your task is to create a summary of these "
                    "relationships. The summary should include the names of the entities involved and a concise synthesis "
                    "of the relationship descriptions. The goal is to capture the most critical and relevant details that "
                    "highlight the nature and significance of each relationship. Ensure that the summary is coherent and "
                    "integrates the information in a way that emphasizes the key aspects of the relationships."
                ),
            ),
            ChatMessage(role="user", content=text),
        ]
        response = OpenAI().chat(messages)
        clean_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return clean_response

    def build_communities(self):
        """Builds communities from the graph and summarizes them."""
        nx_graph = self._create_nx_graph()
        community_hierarchical_clusters = hierarchical_leiden(
            nx_graph, max_cluster_size=self.max_cluster_size
        )
        community_info = self._collect_community_info(
            nx_graph, community_hierarchical_clusters
        )
        self._summarize_communities(community_info)

    def _create_nx_graph(self):
        """Converts internal graph representation to NetworkX graph."""
        nx_graph = nx.Graph()
        for node in self.graph.nodes.values():
            nx_graph.add_node(str(node))
        for relation in self.graph.relations.values():
            nx_graph.add_edge(
                relation.source_id,
                relation.target_id,
                relationship=relation.label,
                description=relation.properties["relationship_description"],
            )
        return nx_graph

    def _collect_community_info(self, nx_graph, clusters):
        """Collect detailed information for each node based on their community."""
        community_mapping = {item.node: item.cluster for item in clusters}
        community_info = {}
        for item in clusters:
            cluster_id = item.cluster
            node = item.node
            if cluster_id not in community_info:
                community_info[cluster_id] = []

            for neighbor in nx_graph.neighbors(node):
                if community_mapping[neighbor] == cluster_id:
                    edge_data = nx_graph.get_edge_data(node, neighbor)
                    if edge_data:
                        detail = f"{node} -> {neighbor} -> {edge_data['relationship']} -> {edge_data['description']}"
                        community_info[cluster_id].append(detail)
        return community_info

    def _summarize_communities(self, community_info):
        """Generate and store summaries for each community."""
        for community_id, details in community_info.items():
            details_text = (
                "\n".join(details) + "."
            )  # Ensure it ends with a period
            self.community_summary[
                community_id
            ] = self.generate_community_summary(details_text)

    def get_community_summaries(self):
        """Returns the community summaries, building them if not already done."""
        if not self.community_summary:
            self.build_communities()
        return self.community_summary

In [24]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex(
    nodes=nodes,
    property_graph_store=GraphRAGStore(),
    kg_extractors=[kg_extractor],
    show_progress=True,
)

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain/chains/api/base.py:56: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import TextRequestsWrapper
/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(mes

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

### <b><font color='darkgreen'>Step 6: Detect Communities and Summarize</font></b>
Use [**graspologic**](https://github.com/graspologic-org/graspologic)’s [**Hierarchical Leiden algorithm**](https://en.wikipedia.org/wiki/Leiden_algorithm) to detect communities and generate summaries. **Communities are groups of nodes** (<font color='brown'>entities</font>) **that are densely connected internally but sparsely connected to other groups. This algorithm maximizes a metric called modularity, which measures the quality of dividing a graph into communities**.

In [25]:
index.property_graph_store.build_communities()

NameError: name 'index' is not defined

### <b><font color='darkgreen'>Step 7: Query the Graph</font></b>
<font size='3ptx'><b>Initialize the <font color='blue'>GraphRAGQueryEngine</font> to query the processed data.</b></font>

When a query is submitted, the engine retrieves relevant community summaries from the <b><font color='blue'>GraphRAGStore</font></b>. For each summary, it uses the LLM to generate a specific answer contextualized to the query via the `generate_answer_from_summary` method. These partial answers are then synthesized into a coherent final response using the `aggregate_answers` method, where the LLM combines multiple perspectives into a concise output.

In [28]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.llms import LLM
class GraphRAGQueryEngine(CustomQueryEngine):
    graph_store: GraphRAGStore
    llm: LLM

    def custom_query(self, query_str: str) -> str:
        """Process all community summaries to generate answers to a specific query."""
        community_summaries = self.graph_store.get_community_summaries()
        community_answers = [
            self.generate_answer_from_summary(community_summary, query_str)
            for _, community_summary in community_summaries.items()
        ]

        final_answer = self.aggregate_answers(community_answers)
        return final_answer

    def generate_answer_from_summary(self, community_summary, query):
        """Generate an answer from a community summary based on a given query using LLM."""
        prompt = (
            f"Given the community summary: {community_summary}, "
            f"how would you answer the following query? Query: {query}"
        )
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content="I need an answer based on the above information.",
            ),
        ]
        response = self.llm.chat(messages)
        cleaned_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return cleaned_response

    def aggregate_answers(self, community_answers):
        """Aggregate individual community answers into a final, coherent response."""
        # intermediate_text = " ".join(community_answers)
        prompt = "Combine the following intermediate answers into a final, concise response."
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content=f"Intermediate answers: {community_answers}",
            ),
        ]
        final_response = self.llm.chat(messages)
        cleaned_final_response = re.sub(
            r"^assistant:\s*", "", str(final_response)
        ).strip()
        return cleaned_final_response

In [27]:
query_engine = GraphRAGQueryEngine(
    graph_store=index.property_graph_store, llm=llm
)
response = query_engine.query("What are news related to financial sector?")
display(Markdown(f"{response.response}"))

NameError: name 'index' is not defined

## <b><font color='darkblue'>Wrapping Up</font></b>
That’s all! I hope you enjoyed reading this article. <font color='green'><b>There’s no doubt that Graph RAG enables you to answer both specific factual and complex abstract questions by understanding the relationships and structures within your data</b></font>. However, <font color='darkred'><b>it’s still in its early stages and has limitations, particularly in terms of token utilization, which is significantly higher than traditional RAG</b></font>. Nevertheless, it’s an important development, and I personally look forward to seeing what’s next. If you have any questions or suggestions, feel free to share them in the comments section below.